# Import data

In [24]:
import pandas
base_data_url = "../datasets/project/road safety/road-safety-data-{}"

In [25]:
accidents_16 = pandas.read_csv(base_data_url.format("accidents-2016.csv"))
accidents_15 = pandas.read_csv(base_data_url.format("accidents-2015.csv"))
accidents_14 = pandas.read_csv(base_data_url.format("accidents-2005-2014.csv"))

In [26]:
accidents = pandas.DataFrame()
accidents = pandas.concat([accidents, accidents_15, accidents_14], axis=0)

In [75]:
# accidents

# Investigate correlations
1. Are serious or fatal accidents more likely in bad weather?
2. Are serious or fatal accidents more likely during the night?
3. Do serious or fatal accidents have, on average, a longer distance from a streetlight?
4. Do any times of day, days of week, or weeks of year have more serious or fatal accidents?

# 1: Bad weather

In [56]:
# Replace the weather with a number: 1 for bad weather, 0 for good
replacement_list = {'Fine + high winds': 1,
  'Fine no high winds': 0,
  'Fog or mist': 1,
  'Other': 0,
  'Raining + high winds': 1,
  'Raining no high winds': 1,
  'Snowing + high winds': 1,
  'Snowing no high winds': 1}

In [62]:
severities = ["Serious", "Slight", "Fatal"]

In [67]:
w_dict = {}
for severity in severities:
    w_dict[severity] = {}
    for weather_type in replacement_list.keys():
        matches = len(accidents[(accidents["Weather_Conditions"] == weather_type) &
                               (accidents["Accident_Severity"] == severity)])
        w_dict[severity][weather_type] = matches

In [68]:
# Plot results
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (40,20)

d_serious = w_dict["Serious"]
d_slight = w_dict["Slight"]
d_fatal = w_dict["Fatal"]


In [70]:
def plot_wtype(wtype, title):
    plt.figure()
    plt.title = title
    plt.bar(range(len(wtype)), list(wtype.values()), align='center')
    plt.xticks(range(len(wtype)), list(wtype.keys()))
    plt.show()

In [74]:
# plot_wtype(d_fatal, "Fatal")

In [20]:
# Replace the weather
# accidents["Weather_Conditions"] = accidents["Weather_Conditions"].map(replacement_list)